In [48]:
import cv2
import numpy as np

In [49]:
img = cv2.imread("mod_img.jpg")
img3 = np.copy(img)
img2 = img
grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_,th = cv2.threshold(grey, 150, 255, cv2.THRESH_BINARY)
kernel = np.ones((1,5),np.uint8)
th = cv2.erode(th, kernel, iterations = 2)
th = cv2.dilate(th, kernel, iterations = 2)
th = 255 - th

nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(th, None, None, None, 4, cv2.CV_32S)
areas = stats[1:,cv2.CC_STAT_AREA]
result = np.zeros((labels.shape), np.uint8)
for i in range(0, nlabels - 1):
    if areas[i] >= 210:   #keep
        result[labels == i + 1] = 255

#cv2.imshow("Binary", binary_map)
result = 255-result

kernel = np.ones((3, 3), np.uint8)   
result = cv2.erode(result, kernel, iterations = 1)
result = cv2.dilate(result, kernel, iterations = 1)

cv2.imwrite("inter1.jpg", result)

True

In [50]:
contours, hierarchy = cv2.findContours(th, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centroid = set()
for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])


i = 0
others = list()
while i<len(boundRect):
    if boundRect[i][2] > 20 and boundRect[i][3] > 20:
        i += 1
    else:
        temp = boundRect[i]
        boundRect.remove(boundRect[i])
        if temp[2] >10 and temp[3] >5 and abs(temp[2] -temp[3])<8:
            others.append(temp)
        
boundRect.sort(key = lambda x:x[0])


i = 0

while i < len(boundRect):
    centroid = (boundRect[i][0] + boundRect[i][2]/2, boundRect[i][1] + boundRect[i][3]/2)
    j = i+1 
    while j < len(boundRect):
        if abs(boundRect[j][0]+boundRect[j][2]/2 -centroid[0])<5 and abs(boundRect[j][1]+boundRect[j][3]/2 -centroid[1])<5: 
            boundRect.remove(boundRect[j])
            break
        else:
                j+=1
    i += 1
print(boundRect)


[(38, 356, 252, 54), (41, 231, 231, 51), (56, 492, 250, 64), (360, 641, 101, 60), (365, 483, 66, 50), (376, 224, 211, 45), (384, 346, 218, 52), (490, 481, 100, 51)]


In [53]:
s = set()
for small in others:
    x, y, w, h = small[0:]
    for i in range(len(boundRect)):
        x1, y1, w1, h1 = boundRect[i][0:]
        if x >= x1+w1-h1 and x+w <= x1+w1 and y >= y1 and y+h <= y1+h1:
            cv2.rectangle(img2, (x1,y1), (x1+w1, y1+h1), (255,0,0), 2)
            cv2.rectangle(img2, (x,y), (x+w, y+h), (70,205,0), 2)
            s.add((x1,y1,w1,h1))
            break
            
for j in boundRect:
    if j in s:
        boundRect.remove(j)

for c in boundRect:
    cv2.rectangle(img2, (int(c[0]), int(c[1])), \
          (int(c[0]+c[2]), int(c[1]+c[3])), (0,0,255), 2)
        
cv2.imwrite("inter2.jpg", img2)     

True